In [1]:
from kafka import KafkaConsumer
import json
from collections import defaultdict
import time

# 1. Kết nối Kafka để lắng nghe
consumer = KafkaConsumer(
    'user_clicks',
    bootstrap_servers=['kafka:29092'],
    auto_offset_reset='latest', # Chỉ nghe tin nhắn mới nhất
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

In [2]:
print("🕵️ ĐANG GIÁM SÁT GIAN LẬN...")

# Bộ nhớ tạm để đếm click (State)
click_counts = defaultdict(int)
start_window = time.time()

🕵️ ĐANG GIÁM SÁT GIAN LẬN...


In [3]:
for msg in consumer:
    event = msg.value
    user_id = event['session']
    
    # --- LOGIC XỬ LÝ (Giả lập Window 10 giây) ---
    
    # 1. Đếm số click
    click_counts[user_id] += 1
    
    # 2. Kiểm tra ngưỡng (Threshold)
    if click_counts[user_id] > 5:
        print(f"🚨 CẢNH BÁO: User {user_id} click quá nhanh! ({click_counts[user_id]} lần) -> CHẶN NGAY!")
    else:
        print(f"✅ User {user_id} - OK ({click_counts[user_id]} lần)")
        
    # 3. Reset cửa sổ sau 10 giây (Tumbling Window)
    if time.time() - start_window > 10:
        print("--- 🔄 Reset Cửa Sổ Đếm ---")
        click_counts.clear()
        start_window = time.time()

✅ User 3 - OK (1 lần)
✅ User 4 - OK (1 lần)
✅ User 3 - OK (2 lần)
✅ User 1 - OK (1 lần)
✅ User 2 - OK (1 lần)
✅ User 5 - OK (1 lần)
✅ User 1 - OK (2 lần)
✅ User 2 - OK (2 lần)
✅ User 1 - OK (3 lần)
✅ User 2 - OK (3 lần)
✅ User 4 - OK (2 lần)
✅ User 1 - OK (4 lần)
✅ User 1 - OK (5 lần)
✅ User 3 - OK (3 lần)
✅ User 5 - OK (2 lần)
🚨 CẢNH BÁO: User 1 click quá nhanh! (6 lần) -> CHẶN NGAY!
🚨 CẢNH BÁO: User 1 click quá nhanh! (7 lần) -> CHẶN NGAY!
✅ User 5 - OK (3 lần)
🚨 CẢNH BÁO: User 1 click quá nhanh! (8 lần) -> CHẶN NGAY!
✅ User 2 - OK (4 lần)
✅ User 5 - OK (4 lần)
--- 🔄 Reset Cửa Sổ Đếm ---
✅ User 4 - OK (1 lần)
✅ User 4 - OK (2 lần)
✅ User 2 - OK (1 lần)
✅ User 3 - OK (1 lần)
✅ User 2 - OK (2 lần)
✅ User 4 - OK (3 lần)
✅ User 4 - OK (4 lần)
✅ User 2 - OK (3 lần)
✅ User 2 - OK (4 lần)
✅ User 4 - OK (5 lần)
✅ User 2 - OK (5 lần)
🚨 CẢNH BÁO: User 4 click quá nhanh! (6 lần) -> CHẶN NGAY!
🚨 CẢNH BÁO: User 4 click quá nhanh! (7 lần) -> CHẶN NGAY!
✅ User 3 - OK (2 lần)
🚨 CẢNH BÁO: User 4 clic

KeyboardInterrupt: 